# Applied linear modeling replication exercise with statsmodels

In [302]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [247]:
# Read in PSID data
psid = pd.read_csv('<PSID>', index_col=False)
psid = psid.rename(columns={'WICpreg': 'wic_participant',
                            'AGE97': 'child_age_97',
                            'faminc97': 'family_income_97',
                            'bthwht': 'birth_weight',
                            'HOME97': 'parenting_prac',
                            'mathraw97': 'child_math_score_97'})
ivs = ['child_math_score_97', 'wic_participant', 'child_age_97', 'family_income_97', 'birth_weight', 'parenting_prac']
psid_filtered = psid[ivs].dropna()
psid_ivs = psid[['wic_participant', 'child_age_97', 'family_income_97', 'birth_weight', 'parenting_prac']]

In [268]:
# Summary statistics
psid_ivs.describe()

,wic_participant,child_age_97,family_income_97,birth_weight,parenting_prac
count,3322.000000,2223.000000,3563.000000,3563.000000,3563.000000
mean,0.433474,7.466937,49841.254516,0.388156,18.923014
std,0.495629,2.931817,49751.071933,0.487399,3.622913
min,0.000000,3.000000,-72296.260000,0.000000,7.000000
25%,0.000000,5.000000,20175.700000,0.000000,16.000000
50%,0.000000,7.000000,39118.440000,0.000000,19.200000
75%,1.000000,10.000000,64494.990000,1.000000,21.800000
max,1.000000,13.000000,784610.590000,1.000000,27.000000


In [269]:
# IV correlations
psid_ivs.corr()

,wic_participant,child_age_97,family_income_97,birth_weight,parenting_prac
wic_participant,1.000000,-0.089849,-0.392974,0.104006,-0.303607
child_age_97,-0.089849,1.000000,0.050128,0.215802,0.197460
family_income_97,-0.392974,0.050128,1.000000,-0.101093,0.303165
birth_weight,0.104006,0.215802,-0.101093,1.000000,0.060776
parenting_prac,-0.303607,0.197460,0.303165,0.060776,1.000000


In [304]:
# Model 1 
model1 = smf.ols('child_math_score_97 ~ wic_participant+child_age_97+family_income_97+birth_weight+parenting_prac',
                data=psid_filtered).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     child_math_score_97   R-squared:                       0.871
Model:                             OLS   Adj. R-squared:                  0.871
Method:                  Least Squares   F-statistic:                     2745.
Date:                 Fri, 25 Sep 2020   Prob (F-statistic):               0.00
Time:                         16:49:23   Log-Likelihood:                -7124.1
No. Observations:                 2036   AIC:                         1.426e+04
Df Residuals:                     2030   BIC:                         1.429e+04
Df Model:                            5                                         
Covariance Type:             nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          -28.2861      1.401    -20.185      0.000     -31.034     -25.538
wic_participant     -1.9766      0.410     -4.819      0.000      -2.781      -1.172
child_age_97         6.8954      0.064    108.096      0.000       6.770       7.020
family_income_97  2.196e-05   3.78e-06      5.812      0.000    1.45e-05    2.94e-05
birth_weight        -1.8581      0.376     -4.945      0.000      -2.595      -1.121
parenting_prac       0.6713      0.068      9.933      0.000       0.539       0.804
==============================================================================
Omnibus:                       74.355   Durbin-Watson:                   1.896
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              160.451
Skew:                          -0.218   Prob(JB):                     1.44e-35
Kurtosis:                       4.304   Cond. No.                     5.94e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.94e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [305]:
# Model 1 transformations 
psid_filtered['child_age_97_trans'] = psid_filtered['child_age_97'].apply(lambda x: (x+1)**2) # child age
psid_filtered['family_income_97_log'] = psid_filtered['family_income_97'].apply(lambda x: np.log(x)) # family income
psid_filtered = psid_filtered[~psid_filtered.isin([np.nan, np.inf, -np.inf]).any(1)]

In [306]:
# Model 2
model2 = smf.ols('child_math_score_97 ~ wic_participant+family_income_97_log+child_age_97_trans+birth_weight+parenting_prac',
                data=psid_filtered).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     child_math_score_97   R-squared:                       0.847
Model:                             OLS   Adj. R-squared:                  0.846
Method:                  Least Squares   F-statistic:                     2239.
Date:                 Fri, 25 Sep 2020   Prob (F-statistic):               0.00
Time:                         16:49:24   Log-Likelihood:                -7302.2
No. Observations:                 2036   AIC:                         1.462e+04
Df Residuals:                     2030   BIC:                         1.465e+04
Df Model:                            5                                         
Covariance Type:             nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              -18.1731      2.403     -7.562      0.000     -22.886     -13.460
wic_participant         -1.8335      0.457     -4.013      0.000      -2.729      -0.938
family_income_97_log     0.8090      0.221      3.665      0.000       0.376       1.242
child_age_97_trans       0.3965      0.004     97.420      0.000       0.388       0.404
birth_weight            -3.1924      0.414     -7.718      0.000      -4.004      -2.381
parenting_prac           0.8061      0.075     10.742      0.000       0.659       0.953
==============================================================================
Omnibus:                       44.516   Durbin-Watson:                   1.907
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.069
Skew:                          -0.110   Prob(JB):                     1.24e-19
Kurtosis:                       3.989   Cond. No.                     1.20e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [307]:
# Outlier removal
psid_influence_summary = model2.get_influence().summary_frame()
psid_filtered_copy = psid_filtered.copy()  # mediate SettingWithCopyWarning
psid_filtered_copy['cooks_d'] = psid_influence_summary.loc[:,'cooks_d']
psid_filtered_with_cd = psid_filtered_copy[psid_filtered_copy['cooks_d'] < 4/1964]

In [308]:
# Model 3: final model
model3 = smf.ols('child_math_score_97 ~ wic_participant+family_income_97_log+child_age_97_trans+birth_weight',
                data=psid_filtered_with_cd).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     child_math_score_97   R-squared:                       0.883
Model:                             OLS   Adj. R-squared:                  0.883
Method:                  Least Squares   F-statistic:                     3646.
Date:                 Fri, 25 Sep 2020   Prob (F-statistic):               0.00
Time:                         16:49:27   Log-Likelihood:                -6696.4
No. Observations:                 1941   AIC:                         1.340e+04
Df Residuals:                     1936   BIC:                         1.343e+04
Df Model:                            4                                         
Covariance Type:             nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              -15.9672      2.279     -7.007      0.000     -20.436     -11.498
wic_participant         -2.3747      0.407     -5.830      0.000      -3.174      -1.576
family_income_97_log     2.0112      0.208      9.684      0.000       1.604       2.419
child_age_97_trans       0.4232      0.004    115.878      0.000       0.416       0.430
birth_weight            -3.2828      0.368     -8.926      0.000      -4.004      -2.561
==============================================================================
Omnibus:                        9.568   Durbin-Watson:                   1.862
Prob(Omnibus):                  0.008   Jarque-Bera (JB):                9.695
Skew:                           0.170   Prob(JB):                      0.00785
Kurtosis:                       2.935   Cond. No.                     1.21e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.21e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""